In [2]:
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/codahead/Fishial/FishialReaserch')
                
import time
import torch
import os
import cv2
import matplotlib.pyplot as plt
import torchvision.models as models
import numpy as np
from torch import nn
from torch.optim import Optimizer, SGD
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from sklearn.neighbors import KDTree
                
from module.classification_package.src.model import EmbeddingModel, Backbone, Model
from module.classification_package.src.dataset import FishialDataset
from module.classification_package.src.dataset import BalancedBatchSampler
from module.classification_package.src.utils import read_json, setup_logger, find_device
from module.classification_package.src.loss_functions import TripletLoss, QuadrupletLoss
from module.classification_package.src.eval import evaluate
from module.classification_package.src.utils import save_checkpoint
from module.classification_package.src.utils import NewPad
                
from PIL import Image
import numpy as np
import random
import sklearn.metrics.pairwise
import scipy.spatial.distance
import copy
import json
import time

np.set_printoptions(precision=20)
torch.set_printoptions(precision=20)

In [31]:
def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

data_transforms = transforms.Compose([
    NewPad(),
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

model = init_model(QuadrupletLoss(), "../ckpt_0.0_0.915_194.ckpt")
model.eval()

Model(
  (embedding_model): EmbeddingModel(
    (backbone): Backbone(
      (resnet): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), s

In [32]:
loader = transforms.Compose([
            NewPad(),
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
data_set_train = FishialDataset(
        json_path="../dataset/data_train.json",
        root_folder="../dataset",
        transform=loader
    )

data_set_val = FishialDataset(
        json_path="../dataset/data_test.json",
        root_folder="../dataset",
        transform=loader
    )

In [34]:
path_data_of_embeddings = '../data_main.json'
data_main = copy.deepcopy(data_set_train.library_name)
start_time = time.time()
for i in range(len(data_set_train)):
    print("Train: {}".format(len(data_set_train) - i), end="\r")
    output = model.embedding_model(data_set_train[i][0].unsqueeze(0)).detach().numpy()
    converted_list = list([str(sd) for sd in output[0]])
    if "vectors" in data_main[int(data_set_train[i][1])]:
        data_main[int(data_set_train[i][1])]["vectors"].append(converted_list)
    else:
        data_main[int(data_set_train[i][1])].update({"vectors": [converted_list]})
    start_time = time.time()

for i in range(len(data_set_val)):
    print("Test: {}".format(len(data_set_val) - i), end="\r")
    output = model.embedding_model(data_set_val[i][0].unsqueeze(0)).detach().numpy()
    converted_list = list([str(sd) for sd in output[0]])
    if "vectors" in data_main[int(data_set_val[i][1])]:
        data_main[int(data_set_val[i][1])]["vectors"].append(converted_list)
    else:
        data_main[int(data_set_val[i][1])].update({"vectors": [converted_list]})

with open(path_data_of_embeddings, 'w') as f:
    json.dump(data_main, f)

In [35]:
def get_data_base_of_embed(path):
    data = read_json(path)
    for i in data:
        for z in range(len(data[i]["vectors"])):
            data[i]["vectors"][z] = [float(val) for val in data[i]["vectors"][z]]
    return data

In [36]:

data_main = get_data_base_of_embed(path_data_of_embeddings)